<a href="https://colab.research.google.com/github/sorrawich-chai/qmsum_on_longformer/blob/main/test_fined_tune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import pipeline

model_checkpoint = "fgiuhsdfkjhfv/longsec3"
sum = pipeline("summarization", model=model_checkpoint,device=0)

In [11]:
!pip install nltk
!git clone https://github.com/Yale-LILY/QMSum.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'QMSum' already exists and is not an empty directory.


In [6]:
import json

# read the dataset
# please enter the path of your data
split = 'test'
data_path = 'QMSum/data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 35 meetings in the test set.


In [7]:
from nltk import word_tokenize
# tokneize a sent
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [8]:
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [9]:
import nltk
nltk.download('punkt')

# process data for BART
# the input of the model here is the gold span corresponding to each query
bart_data_gold = []
for i in range(len(data)):
    # get meeting content
    entire_src = []
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
        cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
        entire_src.append(cur_turn)
    entire_src = ' '.join(entire_src)
    for j in range(len(data[i]['general_query_list'])):
        cur = {}
        query = tokenize(data[i]['general_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['general_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
    for j in range(len(data[i]['specific_query_list'])):
        cur = {}
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        src = []
        # get the content in the gold span for each query
        for span in data[i]['specific_query_list'][j]['relevant_text_span']:
            assert len(span) == 2
            st, ed = int(span[0]), int(span[1])
            for k in range(st, ed + 1):
                cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
                cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
                src.append(cur_turn)
        src = ' '.join(src)
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + src + ' </s>')
        target = tokenize(data[i]['specific_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
        
print('Total {} query-summary pairs in the {} set'.format(len(bart_data_gold), split))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Total 281 query-summary pairs in the test set


In [19]:
bart_data_gold[0]['tgt']

"this meeting was the eleventh evidence session on the children abolition of defense of reasonable publishment wales bill . barry hughes was there to answer questions related to the bill . six main topics that had been discussed or explained . the first one was how the bill protected the children in terms of prosecutions . the second part explained the divergence between regions and the bill 's implication on england . the third part explained how the law was implemented , and the fourth talked about prosecutions and potential alternatives . barry hughes then further explained our-of-court disposals and responded to a specific infrastructure for these cases . in the last part , the meeting turned to discuss a number of unintended consequences related to the bill ."

In [47]:
predict = sum(bart_data_gold[99]['src'])
predict

Your max_length is set to 1024, but your input_length is only 357. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=178)


[{'summary_text': "the civil engineering contractors association would value the growth of the welsh baccalaureate . ed evans thought that employers did n't understand it , but if they did , they would appreciate this . for his sector , especially the contracting sector , which depends very much on pupils who had come through from a less academic background , the institution gave them the ability not to pursue so much of an academic focus , but to test them in terms of the challenges , but also , to learn more directly from industry ."}]

In [28]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
scores = scorer.score(predict[0]['summary_text'],bart_data_gold[99]['tgt'])
scores